In [1]:
%load_ext autoreload
#To update the imported files

In [3]:
%autoreload
import sys
sys.path.insert(0, '../')
from Constants import *
from utilities_xgb import *

ModuleNotFoundError: No module named 'utilities'

In [23]:
selected_features = ["date_forecast", 'direct_rad:W', 'direct_rad_1h:J', 'clear_sky_rad:W',
                     'clear_sky_energy_1h:J', 'diffuse_rad:W', 'sun_elevation:d', 'is_in_shadow:idx', 'is_day:idx']
hyperparameters = {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'objective': 'reg:squarederror', 'subsample': 0.7}

In [24]:
xgb_a = XGB_model(hyperparameters, X_observed_a, X_estimated_a, y_a, selected_features)
xgb_a.fit()
xgb_a.pred()

xgb_b = XGB_model(hyperparameters, X_observed_b, X_estimated_b, y_b, selected_features)
xgb_b.fit()
xgb_b.pred()

xgb_c = XGB_model(hyperparameters, X_observed_c, X_estimated_c, y_c, selected_features)
xgb_c.fit()
xgb_c.pred()

/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

In [25]:
#Evaluate mean asbolute error
mae_a = xgb_a.mae()
print("A: Mean Absolute Error:", mae_a)

mae_b = xgb_b.mae()
print("B: Mean Absolute Error:", mae_b)

mae_c = xgb_c.mae()
print("C: Mean Absolute Error:", mae_c)

A: Mean Absolute Error: 149.87199027078537
B: Mean Absolute Error: 28.185538404414416
C: Mean Absolute Error: 25.30790992879749


In [ ]:
xgb_a.pred(X_test_a)
xgb_b.pred(X_test_b)
xgb_c.pred(X_test_c)

y_pred_xgb = np.concatenate((xgb_a, xgb_b, xgb_c), axis=0)

# Finding hyperparameters

In [48]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'n_estimators': [100, 200, 500],
    'objective': ['reg:squarederror']
}

# Initialize XGBoost Regressor
xg_reg = xgb.XGBRegressor()

# Initialize Grid Search
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=3, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train["pv_measurement"])

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train a new XGBoost model with the best parameters
best_xg_model = xgb.XGBRegressor(**best_params)
best_xg_model.fit(X_train, y_train["pv_measurement"])
# Calculate the Mean Absolute Error
#mae = mean_absolute_error(y_test["pv_measurement"], predictions)
#print(f"Mean Absolute Error of the best model: {mae}")
#Best parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'objective': 'reg:squarederror', 'subsample': 0.7}

Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.5; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.5; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.5; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimat

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [49]:
# Make predictions
predictions = best_xg_model.predict(X_test_real_clean_mean.drop(columns = ["date_forecast"]))
np.save('X_test_c.npy', predictions)